In [ ]:
import sys
import time
import tornado
from transactions import *
from blockchain_state import *
from blocks import *
from miner import Miner
from node import Node
from connections import run_server, remote_connection

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

import nest_asyncio
from cryptography.hazmat.primitives.serialization import load_pem_private_key

In [ ]:
def calculate_sha1_hash(public_key):
    digest = hashes.Hash(hashes.SHA1(),default_backend())
    digest.update(public_key)
    return digest.finalize()

def load_key(filename):
    with open(filename, 'rb') as pem_in:
        pemlines = pem_in.read()
    private_key = load_pem_private_key(pemlines, None, default_backend())
    return private_key

PR_key=load_key("/Users/foulds/code/dsm070-2022-apr/topic_07/miner_private_key.pem")
pub_key=PR_key.public_key()

pub_key_der=pub_key.public_bytes(  
        encoding=serialization.Encoding.DER,  
        format=serialization.PublicFormat.SubjectPublicKeyInfo,  
    )

addr=calculate_sha1_hash(pub_key_der)
trans = create_signed_transaction(PR_key, bytes.fromhex('bcf177e59d90d6c647918a0882c672be8b1fe289'), 100, 10, 0)
trans.verify(
    sender_balance=12660000,
    sender_previous_nonce=-1)

In [ ]:
REMOTE_NODES = ["ws://node.zimcoin.org:46030/"]
node = Node.start("./blocks_trans.sqlite").proxy()

for remote in REMOTE_NODES:
    remote_connection(node, remote)

In [9]:
print('Trying to send transaction...')
node.received_transactions([trans]).get()

# for connection in node.connections.get():
#     print('connection')
#     #connection.send_transactions([trans])

# # print('Going into tornado loop')
# #tornado.ioloop.IOLoop.current().start()

Trying to send transaction...
Transaction can be added to mempool
Sending transactions [<transactions.Transaction object at 0x107620490>]
Sending {"type": "transactions", "transactions": [{"sender_hash": "735c7388347519116daded418c8fe5846b5e5d6d", "recipient_hash": "bcf177e59d90d6c647918a0882c672be8b1fe289", "sender_public_key": "3056301006072a8648ce3d020106052b8104000a03420004b12a1436344f23dfcd472e0f7aac026346620bf937bfde11bf5bd74d0e825815741e7401c507dc9d12e2024f90a9e7f250777a77391ee50c0c0acd8e306a171b", "amount": 100, "fee": 10, "nonce": 0, "signature": "30450221008371d18886a0bc5f8d03966f81f7011df460bfaf81eab8cff078fbb142b85d7502207dc246c7b3500b64c9f4beb04f466d04d07accfbd9f773b80e92705f2be882ab", "txid": "dc97d7d6e9efb6a7c9e03f9010c1dfe09ad896de41b5543eda8105b1d8b0acd1"}]}


Got message {'type': 'transactions', 'transactions': [{'sender_hash': '735c7388347519116daded418c8fe5846b5e5d6d', 'recipient_hash': 'bcf177e59d90d6c647918a0882c672be8b1fe289', 'sender_public_key': '3056301006072a8648ce3d020106052b8104000a03420004b12a1436344f23dfcd472e0f7aac026346620bf937bfde11bf5bd74d0e825815741e7401c507dc9d12e2024f90a9e7f250777a77391ee50c0c0acd8e306a171b', 'amount': 100, 'fee': 10, 'nonce': 0, 'signature': '30450221008371d18886a0bc5f8d03966f81f7011df460bfaf81eab8cff078fbb142b85d7502207dc246c7b3500b64c9f4beb04f466d04d07accfbd9f773b80e92705f2be882ab', 'txid': 'dc97d7d6e9efb6a7c9e03f9010c1dfe09ad896de41b5543eda8105b1d8b0acd1'}]}
Transaction can be added to mempool
Got message {'type': 'blocks', 'blocks': [{'previous': '000000c423ef1fd66971c62f37e8c8bd8a046ea2aa1297a315bdb59ea43a41f0', 'height': 5266, 'miner': 'c9dfeed021097b0752f9bc18642152af5742bb4b', 'transactions': [], 'timestamp': 1660190355, 'difficulty': 8997480, 'block_id': '000000910f709d3cca43bfd21b77a97863631c2